In [4]:
import os
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
import re
import nltk
import requests
import pyLDAvis
import pyLDAvis.gensim_models
from nltk.stem import WordNetLemmatizer
import spacy
import numpy as np

# Make sure to download nltk stopwords if not already
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dnaso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dnaso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# Define paths to transcripts and tags directories
transcripts_dir = '../cleaned_transcripts/'
tags_dir = '../tags/'
csv_path = '../cleaned_results.xlsx'

# Load the CSV file and filter for "related" videos
csv_data = pd.read_excel(csv_path)
related_videos = csv_data[csv_data["related"] == "yes"]

# Load transcripts
transcripts = []
tags = []

# Process each related video based on its video_id
for video_id in related_videos["Video Id"]:
    # Construct paths based on video ID naming conventions
    transcript_file = os.path.join(transcripts_dir, f"{video_id}_captions.txt")
    tag_file = os.path.join(tags_dir, f"{video_id}.txt")
    
    # Read the transcript and tag files if they exist
    try:
        with open(transcript_file, "r", encoding="utf-8") as file:
            transcripts.append(file.read())
        with open(tag_file, "r", encoding="utf-8") as file:
            tags.append(file.read().strip())
    except FileNotFoundError:
        print(f"Files for video ID {video_id} not found, skipping.")

# Combine the filtered data into a DataFrame
data = pd.DataFrame({"video_id": related_videos["Video Id"], "tags": tags, "transcripts": transcripts})
data["text"] = data["tags"] + " " + data["transcripts"]

# Display the first few rows to verify
print(data.head())

       video_id                                               tags  \
0   M0lKXfyJh-I  ['Mikey, Bustos',, 'Pinoy',, 'Filipino',, 'Phi...   
1   cgnBQ9qOqmY  ['Mikey, Bustos',, 'Pinoy',, 'Filipino',, 'Phi...   
2   kgG1oJiE5qs  ['Mikey, Bustos',, 'Pinoy',, 'Filipino',, 'Phi...   
3   LB20skQI1_k  ['Mikey, Bustos',, 'Pinoy',, 'Filipino',, 'Phi...   
51  EKNuv99XeCA  ['Chinese',, 'Sassa, Dagdag',, 'La, Vie, En, R...   

                                          transcripts  \
0   This is Pinoy life,\nHere in the Philippines,\...   
1   I \nam now a total germaphobe,\nAlways disinfe...   
2   Manila, oh nana\nI love to party with my frien...   
3   It's Mikey b*tch!\nAt the palengke (Filipino m...   
51  Hello.my name is max, from china!\nthanks for ...   

                                                 text  
0   ['Mikey, Bustos',, 'Pinoy',, 'Filipino',, 'Phi...  
1   ['Mikey, Bustos',, 'Pinoy',, 'Filipino',, 'Phi...  
2   ['Mikey, Bustos',, 'Pinoy',, 'Filipino',, 'Phi...  
3   ['Mikey,

In [ ]:
# Step 2: Text Preprocessing
stop_words = set(stopwords.words("english"))

nlp = spacy.load("tl_fasttext_transition")

# Function to fetch stopwords from GitHub URL
def fetch_stopwords_from_github(url):
    response = requests.get(url)
    github_stopwords = response.text.splitlines()  # Split by new lines
    return set(github_stopwords)

# GitHub URL for stopwords
github_stopwords_url = 'https://raw.githubusercontent.com/stopwords-iso/stopwords-en/master/stopwords-en.txt'
github_stopwords = fetch_stopwords_from_github(github_stopwords_url)

custom_stop_words = ['like', 'yeah', 'know', 'um', 'uh', 'really', 'one', 'go', 'right', 'okay', 'well', 'said', 
                     'going', 'got', 'na', 'always', 'every', 'each', 'say', 'el', 'little', 'still', 
                     'best', 'dutch', 'nice', 'great', 'awesome', 'good', 'cool', 'love', 'amazing', 'wow' ]
broad_terms = ['philippines', 'philippine', 'british', 'filipino', 'video', 'http', 'korea', 'korean', 
               'youtube', 'google', 'united', 'america', 'american']
kpop_keywords = ['kpop', '필리핀', 'bts', 'blackpink', 'twice', 'exo', 'k-pop', 'seventeen', 
                 'stray kids', 'nct', 'kdrama', 'aespa', 'taehyung', 'jimin', 'jungkook']
more_keywords = [
    'breaking news', 'report', 'coverage', 'investigation', 'interview', 'documentary', 
    'journalist', 'headline', 'reporter', 'current events', 'special report', 
    'analysis', 'documented', 'broadcast', 'reporting', 'v', 'food', 'travel', 'react', 
    'reacts', 'reaction', 'foreigner', 'thing', 'visit', 'dc', 'japan', 'first', 'fast', 
    'asia', 'ang', 'indian', 'thai', 'vietnamese', 'russia', 'gon', 'canada', 'canadian', 'russian', 
    'russia', 'guy', 'lot', 'bit', 'diba', 'ola', 'cuz', 'thai', 'thailand', 'person', 'citizen', 'foreigner', 'foreign', 'foreigners',
    'facebook', 'filipinos', 'filipinas', 'vlog', 'vlogs', 'vlogging', 'hashtag', 'india', 'bro', 'dito', 'people', 'time', 'music', 'gonna', 'life', 
    'lol', 'guys', 'tho', 'cute', 'hmm', 'huh', 'channel', 'subscribe', 'day6', 'mandarin', 'chinese', 'beautiful',
    'chuckles', 'fbe', 'hit', 'laughs', 'yo', 'ka', 'word', 'living', 'boi', 'minimum', 'ya', 'successful', 'perfectly', 'yeap', 
    'wondering', 'fantastic', 'hurry', 'german', 'age', 'country', 'subscribing', 'bluesy', 'jump', 'pretty', 'understanding', 'personalized',
    'and', 'the', 'to', 'of', 'in', 'for', 'on', 'with', 'at', 'by', 'from', 'up', 'about', 'over', 'into', 'through', 'between', 'under', 'against', 'all',
    'you', 'haha', 'hahaha', 'ha', 'hey', 'bye', 'hello', 'hi', 'oh', 'blah', 'easy', 'alright', 'ta', 'day', 'ooh', 'en', 'do', 'lot', 'comment', 'notification', 
    'korean', 'jjajangmyeon', 'jajangmyeon', 'damn', 'yall', 'month', 'week', 'year', 'ohhh', 'pvf', 'dude', 'mmm', 'kagilagilalas', 'ofcourse', 'australia', 'uxo', 
    'atleast', 'yusuf', 'bangkok', 'ot', 'anytime', 'allover', 'kala', 'nope', 'wan', 'brazil', 'smooth', 'ot', 'timeshere', 'batchof', 'yep', 'opo', 'del',
    'gosh', 'po', 'ourself', 'wo', 'wait', 'ugh', 'nyc', 'whoa', 'nicaragua', 'yup', 'em', 'bout', 'le', 'omg', 'overwhelm', 'maam', 'nicer', 'haha', 'hahaha', 'ha', 
    'nbcs', 'lana', 'rc', 'whatsoever', 'oxy', 'decade', 'whyd', 'unknown', 'ahhhhh', 'ohoh', 'ohto', 'ohhhh', 'bruh', 'ooe', 'ahmedabad', 'mexico', 
    'understand', 'excuse', 'kinda', 'applause', 'oooh', 'thiswhat', 'nevermind', 'ahh', 'againthank', 'toto', 'aww', 'nah', 'bbmas', 'ay', 'op', 'huh', 'huhu',
    'tada', 'beacuse', 'voila', 'upstairs', 'thatswhy', 'yea', 'that', 'armenia', 'or', 'not', 'funwhat', 'aka', 'armeniathat', 'woosexy', 'worth', 'laugh', 'box', 
    'xd', 'vb', 'eff', 'ananya', 'welsh', 'latron', 'shout', 'whatwhat', 'what', 'pause', 'why', 'thats', 'byebye', 'iv', 'bye'  
]
stop_words.update(custom_stop_words, kpop_keywords, broad_terms, more_keywords, github_stopwords)

lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase and remove non-alphabet characters
    text = re.sub(r'[^a-z0-9\s]', '', text.lower())
    # Tokenize
    words = word_tokenize(text)
    # Remove stopwords and filter out short words
    words = [word for word in words if word not in stop_words and len(word) > 1 and word.isalpha()]

    # for word in words:
    #     if is_not_filipino_word(word):
    #         words.remove(word)


    words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)

def is_not_filipino_word(word):
    # Process the word using SpaCy
    doc = nlp(word)
    # Use the word vector from SpaCy to determine if it's a Filipino word
    embedding = doc.vector
    return np.linalg.norm(embedding) < 0.25  # This threshold might need adjustment



data["cleaned_text"] = data["text"].apply(preprocess_text)

tfidf = TfidfVectorizer(max_df=0.8, min_df=2, stop_words=custom_stop_words)
temp = data["cleaned_text"]
X = tfidf.fit_transform(temp)

vocabulary = tfidf.get_feature_names_out()
vocabulary

def filter_words(text):
    words = word_tokenize(text) # Split text into words
    print(words)
    filtered_words = [word for word in words if word in vocabulary] 
    print(filtered_words) # Keep only words in vocab
    return " ".join(filtered_words)  # Join back into a string

data["tfidf"] = data["cleaned_text"].apply(filter_words)

In [7]:
# Created File Using only Cleaned Text
with open('cleaned_aggregate.txt', 'w', encoding='utf-8') as file:
    for row in data['cleaned_text']:
        file.write(" ".join(row).replace(',', '') + '\n')


# to use HLTM:
# java -cp HLTA.jar;HLTA-deps.jar tm.hlta.HTD "file name of text" "output name"

In [8]:
# Generate text file for titles only

def clean_tags(text):
    # Lowercase and remove non-alphabet characters
    text = re.sub(r'\W+', ' ', text.lower())
    # Tokenize
    words = word_tokenize(text)
    # Remove stopwords and filter out short words
    words = [word for word in words if word not in stop_words and len(word) > 1 and word.isalpha()]
    words = [lemmatizer.lemmatize(word) for word in words]
    return words

data["cleaned_tags"] = data["tags"].apply(clean_tags)

with open('tags.txt', 'w', encoding='utf-8') as file:
    for row in data['cleaned_tags']:
        file.write(" ".join(row).replace(',', '') + '\n')